In [ ]:
!pip install pyvis

In [13]:
import networkx as nx
import random
import numpy as np
from networkx.algorithms.community import modularity

def initialize_particles(graph, num_particles, num_clusters):
    """Initialize particles with random assignments to clusters."""
    particles = []
    for _ in range(num_particles):
        # Assign each node to a random cluster (0 to num_clusters-1)
        particle = {node: random.randint(0, num_clusters - 1) for node in graph.nodes}
        particles.append(particle)
    return particles

def calculate_fitness(graph, particle):
    """Calculate the modularity of the given particle (cluster assignment)."""
    # Convert particle dictionary into a list of sets for modularity calculation
    clusters = {}
    for node, cluster_id in particle.items():
        if cluster_id not in clusters:
            clusters[cluster_id] = set()
        clusters[cluster_id].add(node)
    return modularity(graph, list(clusters.values()))

def update_velocity(velocity, particle, personal_best, global_best, w, c1, c2):
    """Update the velocity of a particle."""
    new_velocity = {}
    for node in particle:
        r1 = random.random()
        r2 = random.random()

        # Personal and global influence
        personal_influence = c1 * r1 * (personal_best[node] - particle[node])
        global_influence = c2 * r2 * (global_best[node] - particle[node])

        # Update velocity (weighted sum of current velocity, personal, and global influence)
        new_velocity[node] = w * velocity[node] + personal_influence + global_influence

    return new_velocity

def update_position(graph, particle, velocity, num_clusters):
    """Update the position of a particle based on its velocity."""
    new_particle = {}
    for node in particle:
        if random.random() < sigmoid(velocity[node]):
            new_particle[node] = (particle[node] + 1) % num_clusters
        else:
            new_particle[node] = particle[node]
    return new_particle

def sigmoid(x):
    """Sigmoid function to limit the velocity influence."""
    return 1 / (1 + np.exp(-x))

def pso_clustering(graph, num_particles=10, num_clusters=3, iterations=50, w=0.5, c1=1.5, c2=1.5):
    """Cluster a graph using Particle Swarm Optimization (PSO)."""
    # Initialize particles and velocities
    particles = initialize_particles(graph, num_particles, num_clusters)
    velocities = [{node: 0 for node in graph.nodes} for _ in range(num_particles)]

    # Initialize personal bests and global best
    personal_bests = particles[:]
    personal_best_scores = [calculate_fitness(graph, p) for p in particles]
    global_best = personal_bests[np.argmax(personal_best_scores)]
    global_best_score = max(personal_best_scores)

    for _ in range(iterations):
        for i in range(num_particles):
            # Update velocity and position
            velocities[i] = update_velocity(velocities[i], particles[i], personal_bests[i], global_best, w, c1, c2)
            particles[i] = update_position(graph, particles[i], velocities[i], num_clusters)

            # Update personal best
            fitness = calculate_fitness(graph, particles[i])
            if fitness > personal_best_scores[i]:
                personal_bests[i] = particles[i]
                personal_best_scores[i] = fitness

        # Update global best
        best_particle_idx = np.argmax(personal_best_scores)
        if personal_best_scores[best_particle_idx] > global_best_score:
            global_best = personal_bests[best_particle_idx]
            global_best_score = personal_best_scores[best_particle_idx]

    return global_best, global_best_score

import sys
from pyvis.network import Network
project_path = r"c:\Progetti\progetto_asnm"
sys.path.append(project_path)
from Code.notebook.graph.GraphConstructor import GraphConstructor

# Crea il grafo utilizzando GraphConstructor
graph_builder = GraphConstructor()
graph_builder.build_graph()
graph = graph_builder.graph
 

# Numero di cluster desiderati
num_clusters = 3

# Esegui il clustering con PSO
best_partition, best_score = pso_clustering(graph, num_particles=20, num_clusters=num_clusters, iterations=100)

import networkx as nx
# Aggiungi i cluster come attributo dei nodi
gr = graph.copy()


for node in gr.nodes():
    gr.nodes[node].clear()  # Rimuove tutti gli attributi del nodo

for node, cluster in best_partition.items():
    gr.nodes[node]['cluster'] = cluster

# Esporta il grafo in formato GML
nx.write_gml(gr, "graph_with_clusters.gml")

In [17]:
import networkx as nx
# Aggiungi i cluster come attributo dei nodi
gr = graph.copy()


for node in gr.nodes():
    gr.nodes[node].clear()  # Rimuove tutti gli attributi del nodo

for node, cluster in best_partition.items():
    gr.nodes[node]['cluster'] = cluster

# Esporta il grafo in formato GML
nx.write_gml(gr, "graph_with_clusters.gml")